---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    train.csv - the training set (all tickets issued 2004-2011)
    test.csv - the test set (all tickets issued 2012-2016)
    addresses.csv & latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [1]:


def blight_model():
    
    #import matplotlib.pyplot as plt
    import pandas as pd
    import numpy as np
    #from IPython.display import display
    from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
    from sklearn.preprocessing import Imputer,\
                                  LabelEncoder,\
                                  MinMaxScaler,\
                                  OneHotEncoder,\
                                  StandardScaler
                

    from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier

    from sklearn.metrics import auc, roc_auc_score, accuracy_score, classification_report

    from sklearn.preprocessing import Imputer
    
    # loading the addresses
    addresses = pd.read_csv('addresses.csv')
    latlons = pd.read_csv('latlons.csv')
    
    # loading training and test original blight data
    train_data = pd.read_csv('train.csv', encoding = "ISO-8859-1")
    test_data = pd.read_csv('test.csv')
    
    # joining original blight data with the addresses to extract lat and lon
    merged_address = pd.merge(addresses, latlons, on='address', how='left')
    train_data_merged = pd.merge(train_data, merged_address, on='ticket_id', how='left')
    test_data_merged = pd.merge(test_data, merged_address, on='ticket_id', how='left')
    test_data_merged2 = test_data_merged
    
    # selecting the numerical columns to be included in the model
    num_cols = ['fine_amount', 
                'late_fee',
                'discount_amount',
                'judgment_amount',
                'lat',
                'lon']
    
    # selecting the categorical columns to be included in the model
    cat_cols = ['agency_name',
                'inspector_name',
                'violation_code']

    # creating the training and testing data with only the selected columns
    train_data_merged = train_data_merged[num_cols+cat_cols+['compliance']].dropna()
    test_data_merged =  test_data_merged[num_cols+cat_cols]
    
    
    # Imputing nan values in lat and lon columns with the mean values
    imp = Imputer(missing_values=np.nan, strategy='mean')
    imp.fit(test_data_merged[num_cols])
    test_data_num = imp.transform(test_data_merged[num_cols])
    test_data_num = pd.DataFrame(test_data_num, columns=num_cols)
    test_data_merged = pd.concat([test_data_num, 
                                  test_data_merged[cat_cols]],
                                  axis=1)
    
    # combining train and test data into one dataframe
    all_data_merged = train_data_merged.append(test_data_merged)

    # scaling numerical data with a MinMaxScaler
    scaler = MinMaxScaler()
    all_num = pd.DataFrame(scaler.fit_transform(all_data_merged[num_cols]), columns=num_cols)
    
    # performing one hot encoding for categorical columns 
    all_cat = pd.get_dummies(all_data_merged[cat_cols]).reset_index().drop('index', axis=1)
    
    # joining all data after transformations for numerical and categorical data
    all_data_final = pd.concat([all_num, 
                                all_cat,
                                all_data_merged['compliance'].reset_index().drop('index', axis=1)],
                                axis=1)
    
    # Select training and testing data
    train_data = all_data_final[~all_data_final['compliance'].isnull()]
    X_test = all_data_final[all_data_final['compliance'].isnull()]
    X_test.drop('compliance', axis=1, inplace=True)
    
    # splitting training data further into training and validation
    split = StratifiedShuffleSplit(n_splits=1, test_size = 0.2, random_state = 42)
    for train_index, val_index in split.split(train_data, train_data['compliance']):
        # selecting training and validation feature data
        X_train = train_data.loc[train_index]
        X_val = train_data.loc[val_index]
        
        # selecting training and validation target variable
        y_train = X_train['compliance']
        y_val = X_val['compliance']
    
        # dropping the target from training and validation feature data
        X_train.drop('compliance', inplace=True, axis=1)
        X_val.drop('compliance', inplace=True, axis=1)
        
        # selecting the best classifier for the model
        #clf = AdaBoostClassifier() 
        clf = GradientBoostingClassifier()
        #clf = RandomForestClassifier()
        
        # training the model
        clf.fit(X_train, y_train)
        y_pred = clf.predict_proba(X_val)
        #print(classification_report(y_val, clf.predict(X_val)))

    # performing final predictions on the test data
    y_test = clf.predict_proba(X_test)
    
    result = pd.concat([pd.Series(y_test[:,1]),test_data_merged2['ticket_id']],axis=1)\
               .set_index('ticket_id')[0]\
               .rename('compliance') 
    return result

#blight_model()

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2827: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


ticket_id
284932    0.144439
285362    0.029459
285361    0.068955
285338    0.072059
285346    0.090949
285345    0.072059
285347    0.101747
285342    0.337889
285530    0.035553
284989    0.033996
285344    0.075879
285343    0.038179
285340    0.038179
285341    0.075879
285349    0.090949
285348    0.072059
284991    0.033996
285532    0.039879
285406    0.024039
285001    0.056699
285006    0.040568
285405    0.021681
285337    0.024362
285496    0.054990
285497    0.051952
285378    0.029459
285589    0.035010
285585    0.065199
285501    0.054990
285581    0.029459
            ...   
376367    0.024416
376366    0.038034
376362    0.038034
376363    0.047613
376365    0.024416
376364    0.038034
376228    0.058348
376265    0.039578
376286    0.299287
376320    0.039274
376314    0.037573
376327    0.342112
376385    0.329396
376435    0.325438
376370    0.342112
376434    0.058578
376459    0.139330
376478    0.023951
376473    0.039932
376484    0.028200
376482    0.025653
37